In [ ]:
# ranges, instructions = open('./example_input.txt', 'r').read().strip().split("\n\n")
ranges, instructions = open('./input.txt', 'r').read().strip().split("\n\n")

In [ ]:
#cleanup ranges
import polars as pl

fresh_ingredients = (
        pl.DataFrame({"fresh_ingredients": ranges.split("\n")})
        .with_columns(pl.col("fresh_ingredients").str.split("-"))
        .with_columns([
                pl.col("fresh_ingredients").list.get(0).cast(pl.Int64).alias("min"),
                pl.col("fresh_ingredients").list.get(1).cast(pl.Int64).alias("max"),
        ])
        .drop("fresh_ingredients")
        .sort("min")
)


In [ ]:
#cleanup ingredients
ingredients = [int(x.strip()) for x in instructions.split('\n')]

In [ ]:
# %%timeit #51.9 ms ± 2.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
# part 1
is_fresh = [
    fresh_ingredients.select(
        pl.lit(i).is_between(pl.col("min"), pl.col("max")).any()
    ).item()
    for i in ingredients
]
print(sum(is_fresh))

In [ ]:
# %%timeit #56.4 μs ± 969 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
#part 2

# clean up any overlap in the ranges
merged = []
for min, max in fresh_ingredients.iter_rows():
    if not merged: #first iteration
        merged.append([min, max])
    else:
        prev_min, prev_max = merged[-1]
        if min <= prev_max + 1:  # iteration overlapping or next to the previous range i.e [1,3][4,5] or [1,3][2,4]
            merged[-1][1] = max(prev_max, max) #keep the sorted min, but change the max
        else:
            merged.append([min, max])

# Total length
total_length = sum(max - min + 1 for min, max in merged)
print(total_length)